In [21]:
#base
import pandas as pd
import numpy as np 
# 模型
import lightgbm as lgb
from sklearn.model_selection import KFold
# 预处理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook
from tqdm import tqdm
# 效果预测
from sklearn.metrics import precision_score, recall_score


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
test['Attrition'] = -1
data = train.append(test).reset_index(drop=True)

In [8]:
data.info()
# data.describe()

- 
 0   user_id                   1470 non-null   int64 
 1   Age                       1470 non-null   int64 
 2   Attrition                 1470 non-null   object
 3   BusinessTravel            1470 non-null   object
 4   DailyRate                 1470 non-null   int64 
 5   Department                1470 non-null   object
 6   DistanceFromHome          1470 non-null   int64 
 7   Education                 1470 non-null   int64 
 8   EducationField            1470 non-null   object
 9   EmployeeCount             1470 non-null   int64 
 10  EmployeeNumber            1470 non-null   int64 
 11  EnvironmentSatisfaction   1470 non-null   int64 
 12  Gender                    1470 non-null   object
 13  HourlyRate                1470 non-null   int64 
 14  JobInvolvement            1470 non-null   int64 
 15  JobLevel                  1470 non-null   int64 
 16  JobRole                   1470 non-null   object
 17  JobSatisfaction           1470 non-null   int64 
 18  MaritalStatus       

In [12]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Attrition']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))

In [13]:
data.head()

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,No,2,605,2,21,3,1,1,...,3,80,1,29,2,2,1,0,0,0
1,1092,45,No,2,950,1,28,3,5,1,...,4,80,1,8,3,3,5,4,0,3
2,768,40,No,2,300,2,26,3,2,1,...,2,80,1,8,3,2,7,7,7,5
3,569,36,No,0,1434,2,8,4,1,1,...,2,80,0,10,1,3,10,7,0,9
4,911,25,Yes,1,599,2,24,1,1,1,...,4,80,0,1,4,3,1,0,1,0


In [14]:
data.info()

er_id                   1470 non-null   int64 
 1   Age                       1470 non-null   int64 
 2   Attrition                 1470 non-null   object
 3   BusinessTravel            1470 non-null   int32 
 4   DailyRate                 1470 non-null   int64 
 5   Department                1470 non-null   int32 
 6   DistanceFromHome          1470 non-null   int64 
 7   Education                 1470 non-null   int64 
 8   EducationField            1470 non-null   int32 
 9   EmployeeCount             1470 non-null   int64 
 10  EmployeeNumber            1470 non-null   int64 
 11  EnvironmentSatisfaction   1470 non-null   int64 
 12  Gender                    1470 non-null   int32 
 13  HourlyRate                1470 non-null   int64 
 14  JobInvolvement            1470 non-null   int64 
 15  JobLevel                  1470 non-null   int64 
 16  JobRole                   1470 non-null   int32 
 17  JobSatisfaction           1470 non-null   int64 
 18  MaritalStatus             1470

In [15]:
feats = [i for i in data.columns if i not in ['user_id','Attrition']]

In [16]:
feats

['Age',
 'BusinessTravel',
 'DailyRate',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeCount',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'HourlyRate',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'MonthlyRate',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [17]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=70, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'F1',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.002, random_state=2017
    )

In [18]:
data.Attrition.value_counts()

No     988
-1     294
Yes    188
Name: Attrition, dtype: int64

In [19]:
data[data['Attrition']== -1][['user_id']]

,user_id
1176,442
1177,1091
1178,981
1179,785
1180,1332
...,...
1465,1439
1466,481
1467,124
1468,198


In [22]:
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [23]:
data.loc[data['Attrition']==2,'Attrition']=0
train_x = data[data['Attrition']!=-1][feats]
train_y = data[data['Attrition']!=-1]['Attrition']
res=data[data['Attrition']==-1][['user_id']]

In [24]:
test_x= data[data['Attrition']==-1][feats]

In [25]:
res['Attrition'] = 0

In [26]:
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['Attrition'] += model.predict_proba(test_x)[:,1]

]	training's auc: 0.955365	valid_1's auc: 0.749692
[151]	training's auc: 0.955283	valid_1's auc: 0.750616
[152]	training's auc: 0.955324	valid_1's auc: 0.750154
[153]	training's auc: 0.955659	valid_1's auc: 0.749384
[154]	training's auc: 0.95561	valid_1's auc: 0.749384
[155]	training's auc: 0.956043	valid_1's auc: 0.748922
[156]	training's auc: 0.956124	valid_1's auc: 0.748922
[157]	training's auc: 0.956255	valid_1's auc: 0.750616
[158]	training's auc: 0.95632	valid_1's auc: 0.749538
[159]	training's auc: 0.95641	valid_1's auc: 0.748922
[160]	training's auc: 0.956541	valid_1's auc: 0.748153
[161]	training's auc: 0.956688	valid_1's auc: 0.746459
[162]	training's auc: 0.956859	valid_1's auc: 0.746459
[163]	training's auc: 0.956941	valid_1's auc: 0.746305
[164]	training's auc: 0.956933	valid_1's auc: 0.746151
[165]	training's auc: 0.957023	valid_1's auc: 0.746305
[166]	training's auc: 0.957121	valid_1's auc: 0.745844
[167]	training's auc: 0.957284	valid_1's auc: 0.744766
[168]	training's 

In [28]:
res['Attrition'] = res['Attrition']/5

In [29]:
res

,user_id,Attrition
1176,442,0.143020
1177,1091,0.111356
1178,981,0.138595
1179,785,0.156291
1180,1332,0.414322
...,...,...
1465,1439,0.117758
1466,481,0.134461
1467,124,0.247078
1468,198,0.105036


In [30]:
res[['user_id','Attrition']].to_csv('./lgb_baseline.csv', index=False)